
# 🚀 Modelo en Producción con Flask

En esta etapa llevamos nuestro modelo **fuera de Jupyter** y lo convertimos en un servicio accesible desde cualquier aplicación.  
Para lograrlo, construiremos una **API REST** usando *Flask*, que permitirá enviar datos y recibir predicciones del modelo entrenado.



## 🧠 Inferencia fuera del Notebook

Hasta ahora, solo podíamos ejecutar `predict()` dentro de Jupyter.  
Nuestro objetivo es que cualquier sistema (una web, móvil o script) pueda usar el modelo mediante una solicitud HTTP.

👉 Para eso, construiremos una API.


In [ ]:
# Archivo: app.py
from flask import Flask, request, jsonify
import pickle
import numpy as np

# Cargar el modelo entrenado
modelo = None
with open("modelo.pkl", 'rb') as file:
    modelo = pickle.load(file)

# Crear aplicación Flask
app = Flask(__name__)

@app.route('/predecir', methods=['POST'])
def predict():
    # obtener json
    data = request.get_json(force=True)
    
    # convertir los datos a un arreglo de Numpy
    input_data = np.array(data['input']).reshape(1, -1)
    
    # Hacer predicción
    prediccion = modelo.predict(input_data)
    
    # regresar predicción en formato json
    return jsonify({'prediccion': int(prediccion[0])})

if __name__ == '__main__':
    app.run(debug=True)



## ▶️ Ejecución del Servidor

Abre una terminal y ejecuta tu API con:

```bash
python app.py
```

Tu aplicación se ejecutará en:

```
http://127.0.0.1:5000
```

Puedes probar el endpoint `/predecir` con `curl` desde otra terminal:

```bash
curl -X POST http://127.0.0.1:5000/predecir -d '{"input": [0,0,0,0,0,0,0]}'
curl -X POST http://127.0.0.1:5000/predecir -d '{"input": [0,1,0.6159084,0,0,0.55547282,1]}'
```


In [ ]:
import requests

url = "http://127.0.0.1:5000/predecir"
data = {"input": [0,1,0.6159084,0,0,0.55547282,1]}

try:
    response = requests.post(url, json=data, timeout=5)
    if response.status_code == 200:
        print("✅ Respuesta del servidor:", response.json())
    else:
        print(f"⚠️ Error: el servidor respondió con código {response.status_code}")
except requests.exceptions.ConnectionError:
    print("🚫 No se pudo conectar al API. Verifica que Flask esté ejecutándose en el puerto 5000.")
except Exception as e:
    print("⚠️ Ocurrió un error inesperado:", e)



# 🧩 Implementación del Pipeline completo (Notebook 6)

En esta sección continuamos el proyecto integrando **un pipeline completo de Scikit-Learn**, 
que automatiza el preprocesamiento de datos, el escalado y la predicción.  
Esto nos permitirá que el modelo acepte entradas sin transformar directamente desde un JSON más legible.


In [ ]:
# Importar paquetes necesarios
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métricas
from sklearn.metrics import accuracy_score

# Guardado
import pickle


In [ ]:
# Cargar datos limpios
df = pd.read_csv('./data/titanic_clean.csv')
df.head()


In [ ]:
# Definir preprocesamiento
from sklearn.preprocessing import QuantileTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex', 'Embarked']),
        ('age', QuantileTransformer(output_distribution='normal', n_quantiles=500), ['Age']),
        ('fare', QuantileTransformer(output_distribution='normal', n_quantiles=500), ['Fare'])
    ],
    remainder='passthrough'
)


In [ ]:
# Definir los modelos y sus respectivos hiperparámetros
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'model__C': [0.01, 0.1, 1, 10, 100],
            'model__penalty': ['l1', 'l2'],
            'model__solver': ['liblinear', 'saga'],
            'model__max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'model__C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'model__splitter': ['best', 'random'],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4],
            'model__max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'model__n_neighbors': [3, 5, 7]
        }
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3]
        }
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3],
            'model__learning_rate': [0.1, 0.2, 0.3],
            'model__verbose': [-1]
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {
            'model__alpha': [0.1, 1.0, 10.0]
        }
    }
}


In [ ]:
# División de datos
X = df.drop(['Survived'], axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)


In [ ]:
# Variables auxiliares
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}


In [ ]:
# Ciclo for de GridSearch con Pipeline
for nombre, info_modelo in modelos.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('scaler', MinMaxScaler()),
        ('model', info_modelo['modelo'])
    ])
    
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    precision = accuracy_score(y_test, y_pred)
    
    puntajes_modelos.append({'Modelo': nombre, 'Precisión': precision})
    estimadores[nombre] = grid_search.best_estimator_
    
    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_


In [ ]:
# Mostrar resultados y guardar el mejor modelo
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

# Guardar pipeline
with open('pipeline.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)



## ⚠️ Sobre los *Warnings*
Es posible que aparezcan mensajes como:

> The max_iter was reached which means the coef_ did not converge  
> The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6.

Esto **no son errores**. Simplemente indican que algunos hiperparámetros no convergieron en ciertas combinaciones.



# 🔁 Refactorización del API para usar el Pipeline

Actualizaremos el `app.py` para que acepte JSONs con datos sin transformar, 
y use directamente el pipeline guardado (`pipeline.pkl`).


In [ ]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd

app = Flask(__name__)

# Cargar el pipeline
with open('pipeline.pkl', 'rb') as archivo_modelo:
    modelo = pickle.load(archivo_modelo)

@app.route('/predecir', methods=['POST'])
def predecir():
    data = request.get_json()
    input_data = pd.DataFrame([data])
    prediccion = modelo.predict(input_data)
    return jsonify({'Survived': int(prediccion[0])})

if __name__ == '__main__':
    app.run(debug=True)



### ✅ Ejemplo de prueba con cURL

```bash
curl -X POST http://127.0.0.1:5000/predecir -H 'Content-Type: application/json' -d '{
  "Pclass": 2,
  "Sex": "male",
  "Age": 46,
  "SibSp": 0,
  "Parch": 0,
  "Fare": 7.2500,
  "Embarked": "C"
}'
```


In [ ]:
# Prueba local usando requests.post()
import requests

url = "http://127.0.0.1:5000/predecir"
data = {
    "Pclass": 2,
    "Sex": "male",
    "Age": 46,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "C"
}

try:
    response = requests.post(url, json=data, timeout=5)
    if response.status_code == 200:
        print("✅ Respuesta del servidor:", response.json())
    else:
        print(f"⚠️ Código de respuesta inesperado: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("🚫 No se pudo conectar al servidor Flask. Asegúrate de que esté ejecutándose.")
except Exception as e:
    print("⚠️ Error inesperado:", e)
